In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

C:\Users\camdi\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\camdi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\camdi\AppData\Local\Temp/ipykernel_9336/2660306102.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\camdi\anaconda3\l

In [4]:
data = pd.read_csv("nfl_plays_offense_personnel.csv")

In [5]:
data.dropna(subset=['down'], inplace=True)
data.dropna(subset=['play_type'], inplace=True)
data.dropna(subset=['score_differential'], inplace=True)

In [6]:
def is_successful_play(row):
    play_type = row['play_type']
    down = row['down']
    yards_to_go = row['ydstogo']
    yards_gained = row['yards_gained']
    touchdown = row['touchdown']
    score_differential = row['score_differential']
    half_seconds_remaining = row['half_seconds_remaining']
    field_goal_result= row['field_goal_result']
    kick_distance = row['kick_distance']
    quarter = row['qtr']
    ### FIX TIME REMAINING TO SECOND IN QRT AND ADD A FIELD GOAL MADE TO 1,2,3 down and 2,4 qrt with less 60 secs left
    if down == 4:
        if touchdown == 1 or (float(yards_to_go) - float(yards_gained) <= 0) or (play_type == 'punt' and kick_distance >= 40) or (play_type == 'field_goal' and field_goal_result == 'made'):
            return "yes"
    elif down == 3:
        if touchdown == 1 or (float(yards_to_go) - float(yards_gained) <= 0) or (play_type == 'run' and score_differential > 0 and half_seconds_remaining < 300) or ((play_type == 'qb_kneel' or play_type == 'qb_spike') and half_seconds_remaining <= 120) or (play_type == 'field_goal' and field_goal_result == 'made'):
            return "yes"
    elif down == 2:
        if touchdown == 1 or (float(yards_gained)/float(yards_to_go) >= 1.0/2.0) or (play_type == 'run' and score_differential > 0 and half_seconds_remaining < 300) or ((play_type == 'qb_kneel' or play_type == 'qb_spike') and half_seconds_remaining <= 120) or (play_type == 'field_goal' and field_goal_result == 'made'):
            return "yes"
    elif down == 1:
        if touchdown == 1 or (float(yards_gained)/float(yards_to_go) >= 1.0/3.0) or (play_type == 'run' and score_differential > 0 and half_seconds_remaining < 300) or ((play_type == 'qb_kneel' or play_type == 'qb_spike')  and half_seconds_remaining <= 120) or (play_type == 'field_goal' and field_goal_result == 'made'):
            return "yes"
    return "no"

# Apply the function to each row of the dataframe and store the result in a new column
data['success'] = data.apply(is_successful_play, axis=1)
data

,season_type,posteam,posteam_type,side_of_field,yardline_100,quarter_seconds_remaining,half_seconds_remaining,qtr,down,time,...,fumble_lost,touchdown,field_goal_attempt,punt_attempt,offense_formation,offense_personnel,route,o_personnel,personnel_num,success
1,REG,TEN,home,TEN,75,900,1800,1,1.0,15:00,...,0,0,0,0,SINGLEBACK,"1 RB, 2 TE, 2 WR",NaN,"1 RB, 2 TE, 2 WR",12.0,no
2,REG,TEN,home,TEN,78,863,1763,1,2.0,14:23,...,0,0,0,0,SHOTGUN,"1 RB, 2 TE, 2 WR",ANGLE,"1 RB, 2 TE, 2 WR",12.0,no
3,REG,TEN,home,TEN,75,822,1722,1,3.0,13:42,...,0,0,0,0,SHOTGUN,"1 RB, 1 TE, 3 WR",HITCH,"1 RB, 1 TE, 3 WR",11.0,no
4,REG,TEN,home,TEN,75,819,1719,1,4.0,13:39,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,yes
5,REG,ARI,away,ARI,61,807,1707,1,1.0,13:27,...,0,0,0,0,SHOTGUN,"1 RB, 2 TE, 2 WR",GO,"1 RB, 2 TE, 2 WR",12.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130819,POST,KC,home,SF,37,168,168,5,2.0,02:48,...,0,0,0,0,SHOTGUN,"1 RB, 1 TE, 3 WR",SCREEN,"1 RB, 1 TE, 3 WR",11.0,yes
130820,POST,KC,home,SF,32,127,127,5,3.0,02:07,...,0,0,0,0,SHOTGUN,"1 RB, 2 TE, 2 WR",NaN,"1 RB, 2 TE, 2 WR",12.0,yes
130821,POST,KC,home,SF,13,86,86,5,1.0,01:26,...,0,0,0,0,SHOTGUN,"1 RB, 3 TE, 1 WR",NaN,"1 RB, 3 TE, 1 WR",13.0,no
130822,POST,KC,home,SF,10,50,50,5,2.0,00:50,...,0,0,0,0,SHOTGUN,"1 RB, 1 TE, 3 WR",SCREEN,"1 RB, 1 TE, 3 WR",11.0,yes


In [20]:
data_down_dict = {}

for unique_down in data['down'].unique():
    data_down_name = f"data_down{int(unique_down)}"
    data_down_dict[data_down_name] = data[data['down'] == unique_down]
    data_predictors = ['yardline_100', 
                       'half_seconds_remaining', 
                       'ydstogo',
                       'score_differential']

    n_clusters = 200  
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    data_down_dict[data_down_name]['cluster'] = kmeans.fit_predict(data_down_dict[data_down_name][data_predictors])

C:\Users\camdi\AppData\Local\Temp/ipykernel_9336/2091283561.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_down_dict[data_down_name]['cluster'] = kmeans.fit_predict(data_down_dict[data_down_name][data_predictors])
C:\Users\camdi\AppData\Local\Temp/ipykernel_9336/2091283561.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_down_dict[data_down_name]['cluster'] = kmeans.fit_predict(data_down_dict[data_down_name][data_predictors])
C:\Users\camdi\AppData\Local\Temp/ipykernel_9336/2091283561.py

In [26]:
def get_play_success(down, data_predictors):
    if down == 1:
        down_data = data_down_dict['data_down1']
    elif down == 2:
        down_data = data_down_dict['data_down2']
    elif down == 3:
        down_data = data_down_dict['data_down3']
    elif down == 4:
        down_data = data_down_dict['data_down4']
        
        
    cluster_success_rates = {}
    for cluster_id in range(192):
        cluster_data = down_data[down_data['cluster'] == cluster_id]
        success_rate = cluster_data.groupby('play_type')['success'].apply(lambda x: (x == 'yes').mean() * 100)
        cluster_success_rates[cluster_id] = dict(success_rate)

    predicted_cluster = kmeans.predict([data_predictors])
    predicted_cluster[0]
    print(cluster_success_rates[predicted_cluster[0]])

get_play_success(2, ['35','650', '9', '-3'])  

{'pass': 48.484848484848484, 'run': 65.625}
